<a href="https://colab.research.google.com/github/yehezkielgunawan/thesis-risk-classification/blob/main/Create_Custom_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Word Embedding Dataset: https://docs.google.com/spreadsheets/d/1MStLeH0IIoThlNPq_P_0WD1IVyy_QdU_/edit?usp=sharing&ouid=104481541349642552597&rtpof=true&sd=true

In [3]:
import pandas as pd
df = pd.read_excel('/content/gdrive/My Drive/Tesis/EmbeddingDataset.xlsx')
df['content']

0       Sorry, including the expensive price + apk usi...
1       Verifikasi terlalu ribet pending terus, udah n...
2                             menginap yang nyaman sekali
3                                     Ga bisa masuk apk ?
4       kenapa apk boboboxx saya stuck di loading scre...
                              ...                        
2744                                             good job
2745                                                   ok
2746                                            ok banget
2747            Sangat Memuaskan... Balek kampung ahhh...
2748    setelah update kok malah susah, bahkan utk men...
Name: content, Length: 2749, dtype: object

In [4]:
df.head(2).to_json()

'{"Unnamed: 0":{"0":0,"1":1},"content":{"0":"Sorry, including the expensive price + apk using English for people who don\'t understand, it will be difficult to book","1":"Verifikasi terlalu ribet pending terus, udah nyoba pake KTP, SIM juga tetep aja pending, tolong perbaiki lagi"}}'

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

model_name = 'bert-base-multilingual-cased'  # Pre-trained BERT model for multilingual support
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

sentences = df['content'].tolist()

# Tokenize the sentences and convert them into input tensors
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Configure GPU acceleration if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Generate embeddings in batches
batch_size = 16
embeddings = []
with torch.no_grad():
    for i in range(0, len(sentences), batch_size):
        batch_input_ids = input_ids[i:i+batch_size]
        batch_attention_mask = attention_mask[i:i+batch_size]
        batch_input_ids = batch_input_ids.to(device)
        batch_attention_mask = batch_attention_mask.to(device)

        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.extend(batch_embeddings)

embeddings = np.stack(embeddings)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# Define the output file path
output_file = "/content/gdrive/My Drive/Tesis/bert_embeddings.npy"

# Save the embeddings to the output file
np.save(output_file, embeddings)
